Demonstrates the usage of django2pydantic library.

In [ ]:
# Set up the Django environment for the example
from examples.django_setup import setup_django_environment

setup_django_environment()

Lets define some Django models for an example library system:

In [2]:
from django.contrib.auth.models import User
from django.db import models


class Author(models.Model):
    name = models.CharField(max_length=100)
    birth_date = models.DateField()

    class Meta:
        app_label = "tests"

class Publisher(models.Model):
    name = models.CharField(max_length=100)
    address = models.TextField(help_text="Publisher's official address")

    class Meta:
        app_label = "tests"


class Book(models.Model):
    title = models.CharField(max_length=200)
    isbn = models.CharField(max_length=13, unique=True)
    publication_date = models.DateField()
    authors = models.ManyToManyField(Author)  # Many-to-Many relationship
    publisher = models.ForeignKey(Publisher, on_delete=models.CASCADE)  # Foreign Key relationship

    class Meta:
        app_label = "tests"
        default_related_name = "books"

    @property
    def author_names(self) -> str:
        """All authors' names separated by comma."""
        return ", ".join([author.name for author in self.authors.all()])


class Library(models.Model):
    name = models.CharField(max_length=100)
    address = models.TextField()

    class Meta:
        app_label = "tests"


class BookCopy(models.Model):
    book = models.ForeignKey(Book, on_delete=models.CASCADE)  # Foreign Key relationship
    library = models.ForeignKey(Library, on_delete=models.CASCADE)  # Foreign Key relationship
    inventory_number = models.CharField(max_length=20, unique=True)

    class Meta:
        app_label = "tests"
        default_related_name = "book_copies"
        verbose_name_plural = "Book copies"


class Borrowing(models.Model):
    user = models.ForeignKey(User, on_delete=models.CASCADE)  # Foreign Key relationship
    book_copy = models.ForeignKey(BookCopy, on_delete=models.CASCADE)  # Foreign Key relationship
    borrow_date = models.DateField()
    return_date = models.DateField(null=True, blank=True)

    class Meta:
        app_label = "tests"
        default_related_name = "borrowings"


Now lets do some testing:

In [3]:
from django2pydantic import BaseSchema, Infer, InferExcept, SchemaConfig


class BookSchema(BaseSchema[Book]):
    """Book schema example with nested fields."""

    config = SchemaConfig[Book](
        model=Book,
        fields= {
            "title": Infer,
            "isbn": Infer,
            "publication_date": Infer,
            "authors": {"name": InferExcept(description="Book author name")},
            "publisher": {"name": Infer},
            "book_copies": {"library": Infer}, # note: here we use a reverse relation
            "author_names": Infer,  # note: here we use the model's @property method
        }
    )


Now we can print out the JSON schema for the Book model:

In [4]:
from rich import print_json

print_json(data=BookSchema.model_json_schema())

{
  "$defs": {
    "BookSchema_authors": {
      "properties": {
        "name": {
          "description": "Book author name",
          "maxLength": 100,
          "minLength": 1,
          "title": "name",
          "type": "string"
        }
      },
      "required": [
        "name"
      ],
      "title": "BookSchema_authors",
      "type": "object"
    },
    "BookSchema_book_copies": {
      "properties": {
        "library": {
          "description": "library",
          "title": "library",
          "type": "integer"
        }
      },
      "required": [
        "library"
      ],
      "title": "BookSchema_book_copies",
      "type": "object"
    },
    "BookSchema_publisher": {
      "properties": {
        "name": {
          "description": "name",
          "maxLength": 100,
          "minLength": 1,
          "title": "name",
          "type": "string"
        }
      },
      "required": [
        "name"
      ],
      "title": "BookSchema_publisher",
      "type": "object"
    }
  },
  "properties": {
    "title": {
      "description": "title",
      "maxLength": 200,
      "minLength": 1,
      "title": "title",
      "type": "string"
    },
    "isbn": {
      "description": "isbn",
      "maxLength": 13,
      "minLength": 1,
      "title": "isbn",
      "type": "string"
    },
    "publication_date": {
      "description": "publication date",
      "format": "date",
      "title": "publication date",
      "type": "string"
    },
    "authors": {
      "anyOf": [
        {
          "items": {
            "$ref": "#/$defs/BookSchema_authors"
          },
          "type": "array"
        },
        {
          "type": "null"
        }
      ],
      "default": null,
      "description": "authors",
      "title": "authors"
    },
    "publisher": {
      "$ref": "#/$defs/BookSchema_publisher",
      "description": "publisher",
      "title": "publisher"
    },
    "book_copies": {
      "anyOf": [
        {
          "items": {
            "$ref": "#/$defs/BookSchema_book_copies"
          },
          "type": "array"
        },
        {
          "type": "null"
        }
      ],
      "default": null,
      "title": "Book copies"
    },
    "author_names": {
      "description": "All authors' names separated by comma.",
      "title": "Author Names",
      "type": "string"
    }
  },
  "required": [
    "title",
    "isbn",
    "publication_date",
    "publisher",
    "author_names"
  ],
  "title": "BookSchema",
  "type": "object"
}